# X-ray Luminosity - Temperature Relation

In this notebook we make a simple comparison of X-ray luminosities and temperatures to literature. The luminosities and temperatures have been measured during the course of this work. Our data are such that it is not particularly helpful to fit our own scaling relation, but we can overplot them on top of previously measured X-ray luminosity to temperature scaling relations.

## Import Statements

In [1]:
import numpy as np
from astropy.units import Quantity
import pandas as pd

from xga.products.relation import ScalingRelation
from xga.relations.clusters.LT import xcs_sdss_r500_52, xcs_sdss_r2500_52
from xga.models.misc import power_law
from xga.utils import DEFAULT_COSMO

%matplotlib inline

/Users/dt237/code/XGA/xga/utils.py:39: DeprecationWarning: The XGA 'find_all_wcs' function should be imported from imagetools.misc, in the future it will be removed from utils.
  warn(message, DeprecationWarning)
/Users/dt237/code/XGA/xga/utils.py:632: UserWarning: No eSASS installation detected on system, as such all functions in xga.generate.esass will not work.
  warn("No eSASS installation detected on system, as such all functions in xga.generate.esass will not work.")
/opt/anaconda3/envs/xga_dev/lib/python3.8/site-packages/PyAbel-0.8.4-py3.8.egg/abel/tools/circularize.py:4: DeprecationWarning: Please use `map_coordinates` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import map_coordinates
/opt/anaconda3/envs/xga_dev/lib/python3.8/site-packages/PyAbel-0.8.4-py3.8.egg/abel/tools/polar.py:10: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` 

ImportError: cannot import name 'xcs_sdss_r2500_52' from 'xga.relations.clusters.LT' (/Users/dt237/code/XGA/xga/relations/clusters/LT.py)

## Loading data table

We load our measurements of X-ray properties. We choose the 'standard mode' pipeline run (luminosities are broadly consistent with other modes for this sample), with measurements made within $R_{500}$ (as were those of the scaling relations we compare to):

In [ ]:
xray_prop = pd.read_csv("../../outputs/results/ltr_r500_pipeline_results.csv")
xray_prop

We also read in the results for measurements within $R_{2500}$:

In [ ]:
r2500_xray_prop = pd.read_csv("../../outputs/results/ltr_r2500_pipeline_results.csv")
r2500_xray_prop

## Comparing to scaling relations

Here we compare our measurements to existing scaling relations between X-ray luminosity and temperature. The selected scaling relations were not necessarily measured specifically for galaxy groups.

### Reading out data points

We need to get the data points ready for over-plotting on the scaling relations. This involves reading them out of the data table and putting them in astropy quantities. We make sure to multiply the luminosity with the inverse E(z) factor calculated from the cluster redshifts.

In [ ]:
inv_e = DEFAULT_COSMO.inv_efunc(xray_prop['redshift'].values)
lx = Quantity(xray_prop[['Lx500_0.5-2.0', 'Lx500_0.5-2.0-', 'Lx500_0.5-2.0+']].values, 
              'erg/s')*inv_e[..., None]

tx = Quantity(xray_prop[['Tx500', 'Tx500-', 'Tx500+']].values, 'keV')

We go through the same process for out $R_{2500}$ measurements. This includes calculating inverse E(z), as it is possible that the group entries in the second table will be different than in the first (that shouldn't be the case, but why take the risk?):

In [ ]:
r2500_inv_e = DEFAULT_COSMO.inv_efunc(r2500_xray_prop['redshift'].values)
r2500_lx = Quantity(r2500_xray_prop[['Lx2500_0.5-2.0', 'Lx2500_0.5-2.0-', 'Lx2500_0.5-2.0+']].values, 
                    'erg/s')*inv_e[..., None]

r2500_tx = Quantity(r2500_xray_prop[['Tx2500', 'Tx2500-', 'Tx2500+']].values, 'keV')

### XCS cluster scaling relation

This relation was measured by [Giles et al. 2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.516.3878G/abstract), and a version is built into XGA. We trick the existing relation somewhat by adding data points, for the sake of convenience. This relation was not measured specifically from a galaxy group sample, and as such may not make for a perfect comparison (some results indicate somewhat different relations for galaxy groups and clusters). It is still a reasonable place to start however:

In [ ]:
xcs_sdss_r500_52._y_data = lx[:, 0]
xcs_sdss_r500_52._y_err = lx[:, 1]

xcs_sdss_r500_52._x_data = tx[:, 0]
xcs_sdss_r500_52._x_err = tx[:, 1]

xcs_sdss_r500_52._point_names = xray_prop['name'].values

xcs_sdss_r500_52._x_name = r"$T_{\rm{X,500}}$"
xcs_sdss_r500_52._y_name = r"$E(z)^{-1}L^{0.5-2.0}_{\rm{X,500}}$"
xcs_sdss_r500_52.model_colour = 'darkcyan'

That same work produced this $R_{2500}$ scaling relation, which we will also compare our measurements to:

In [ ]:
xcs_sdss_r2500_52._y_data = r2500_lx[:, 0]
xcs_sdss_r2500_52._y_err = r2500_lx[:, 1]

xcs_sdss_r2500_52._x_data = r2500_tx[:, 0]
xcs_sdss_r2500_52._x_err = r2500_tx[:, 1]

xcs_sdss_r2500_52._point_names = r2500_xray_prop['name'].values

xcs_sdss_r2500_52._x_name = r"$T_{\rm{X,2500}}$"
xcs_sdss_r2500_52._y_name = r"$E(z)^{-1}L^{0.5-2.0}_{\rm{X,2500}}$"
xcs_sdss_r2500_52.model_colour = 'darkcyan'

## Plotting our data

Here we make use of the scaling relation objects that we just defined to illustrate how our data compare to the literature relations.

### $R_{500}$

First, we compare our $R_{500}$ measurements - given the low signal-to-noise of our data we may find that these are less trustworthy than $R_{2500}$ measurements, but we have to check.

#### Labelling our data points for context

First of all, we plot the SDSSRM-XCS scaling relation with data points labelled - so it is easy to link particular points to particular groups/clusters:

In [ ]:
xcs_sdss_r500_52.point_names

In [ ]:
xcs_sdss_r500_52.view(x_lims=Quantity([0.3, 5], 'keV'), log_scale=True, label_points=True, conf_level=68, 
                      plot_title='', figsize=(8.5, 8), point_label_offset=(0.005, -0.03), 
                      save_path="../../outputs/figures/tx_lx_relation_comp_pntlab.pdf")

#### Comparing data to the relation without labels

This not only plots the scaling relation, but also overlays our data points. The plot is displayed in the notebook and also saved to disk:

In [ ]:
xcs_sdss_r500_52.view(x_lims=Quantity([0.3, 5], 'keV'), log_scale=True, label_points=False, conf_level=68, 
                      plot_title='', figsize=(8.5, 8), save_path="../../outputs/figures/tx_lx_relation_comp.pdf")

### $R_{2500}$

Now we compare our $R_{2500}$ measurements.

#### Labelling our data points for context

First of all, we plot the SDSSRM-XCS scaling relation with data points labelled - so it is easy to link particular points to particular groups/clusters:

In [ ]:
xcs_sdss_r2500_52.point_names

In [ ]:
# x_lims=Quantity([0.3, 5], 'keV')
xcs_sdss_r2500_52.view(, log_scale=True, label_points=True, conf_level=68, 
                       plot_title='', figsize=(8.5, 8), point_label_offset=(0.005, -0.03), 
                       save_path="../../outputs/figures/tx_lx_r2500_relation_comp_pntlab.pdf")

#### Comparing data to the relation without labels

This not only plots the scaling relation, but also overlays our data points. The plot is displayed in the notebook and also saved to disk:

In [ ]:
xcs_sdss_r2500_52.view(x_lims=Quantity([0.3, 5], 'keV'), log_scale=True, label_points=False, conf_level=68, 
                       plot_title='', figsize=(8.5, 8), 
                       save_path="../../outputs/figures/tx_lx_r2500_relation_comp.pdf")